In [30]:
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from matplotlib import pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

import bo_methods_lib
from bo_methods_lib.bo_functions_generic import gen_theta_set

from bo_methods_lib.CS2_bo_plotters import save_csv, plot_obj_abs_min, value_plotter, plot_obj, plot_Theta, plot_Theta_min,path_name, plot_org_train, save_fig, plot_EI_abs_max

from bo_methods_lib.CS2_bo_functions_multi_dim import set_ep

In [2]:
#https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html

def csv_to_array(csv_path):
    """
    Turn a csv file into a numpy array so that it can be used with bo_plotters
    
    Parameters:
    -----------
        csv_path: str, path of csv you want to turn into an array
    Returns:
    --------
        csv_array: np.ndarray, array on values in the CSV file
    """
    
    csv_pd = pd.read_csv(csv_path, index_col = False)
    csv_pd.drop(columns=csv_pd.columns[0], 
        axis=1, 
        inplace=True)
    csv_array = csv_pd.to_numpy()
    
    return csv_array

## Define Problem Whose Data you want

In [69]:
# DateTime = "2022/10/21/13-05" #Date and Time - Dispayed in job file
DateTime = "2022/10/21/13-18" #Date and Time - Dispayed in job file
DateTime = "../2023/05/10/09-16" #Date and Time - Dispayed in job file #hps = trained SF experiment
DateTime = "../2023/05/09/11-58" #Date and Time - Dispayed in job file 
DateTime = "../2023/05/15/14-39" #Date and Time - Dispayed in job file #hps = 1 SF experiment
DateTime = "../2023/05/22/14-27" #Date and Time - Dispayed in job file #Compare hp trained/1 for 15 iters experiment
# DateTime = None

emulator = False
obj = "LN_obj"
sparse_grid = False
normalize = False
sep_fact = 1.0

exp_d = 1
n = 5

skip_param_types = 0
param_dict = {0 : '\\theta_1', 1 : '\\theta_2'}
bounds_x = np.array([[-2], [2]])
bounds_p = np.array([[-2, -2],
                         [ 2,  2]])
indecies = [list(param_dict.keys())[list(param_dict.values()).index('\\theta_1')],
            list(param_dict.keys())[list(param_dict.values()).index('\\theta_2')]]

Theta_True = np.array([1,-1])

bounds = np.array([[-2, -2],
                   [ 2,  2]])

Bound_Cut = False
denseX = False

param_names_list = [param_dict[0], param_dict[1]] #Fill in depending on problem
mesh_combo = str(param_names_list[0]) + "-" + str(param_names_list[1])

# run = 1
# Bo_iter = 1

tot_iter = bo_iters= 15 #Total Number of iters - Dispayed in job file
tot_runs = 1 #Total Number of runs - Dispayed in job file

run = 1
Bo_iter = 1

set_lengthscale = 1
save_CSV = False

t = 20
if emulator == True:
    t = n*t
    
if Bound_Cut == True:
    cut_bounds = '_cut_bounds'
else:
    cut_bounds = ""

if denseX == True:
    dense = "_dense"
else:
    dense = ""

#Pull Experimental data from CSV
exp_data_doc = '../Input_CSVs/Exp_Data/d='+str(exp_d)+'/n='+str(n)+cut_bounds+dense+'.csv'
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1:exp_d+1]
Yexp = exp_data[:,-1]

#Define GP Testing space
p=20
d = len(Theta_True)
LHS = True
# Need to redefine to save theta_set_org 
# theta_set_org = gen_theta_set(LHS = LHS, n_points = p, dimensions = d, bounds = bounds_p, seed =1)
# Theta1_lin = np.linspace(np.min(theta_set_org[:,indecies[0]]),np.max(theta_set_org[:,indecies[0]]), p)
# Theta2_lin = np.linspace(np.min(theta_set_org[:,indecies[1]]),np.max(theta_set_org[:,indecies[1]]), p)
# theta_mesh = np.array(np.meshgrid(Theta1_lin, Theta2_lin)) 

theta_mesh = gen_theta_set(LHS = False, n_points = p, dimensions = d, bounds = bounds)
theta_mesh = theta_mesh.T.reshape((2, p, p))
# print(theta_mesh)
    
# ep = set_ep(emulator, obj, sparse_grid)
# print(ep)
ep = 1

# tot_iter = bo_iters= 100 #Total Number of iters - Dispayed in job file
# tot_runs = 15 #Total Number of runs - Dispayed in job file

is_figure = False
save_figure = False

run -= 1 # run should be number you want -1
Bo_iter -= 1 #Iter you want -1

In [70]:
#Define method 1A - 2C

if emulator == False:
    if obj == "obj":
        Method = "1A"
    else:
        Method = "1B"
else:
    if obj == "obj":
        if sparse_grid == True:
            Method = "2C"
        else:
            Method = "2B"
    else:
        Method = "2C"
        
print(Method)

1B


## Make a dataframe of EI component value

In [71]:
# Exp_Preds = [Xexp[:,x] for x in range(m)] + [y_sim_Xexp, GP_mean_Xexp, GP_stdev_Xexp, APE_Exp_Preds]
#             #Convert list to array
#             Exp_Preds = np.array( Exp_Preds )
#             #Turn array into pandas df

#Set Properties of EI component path names
fxn = "value_plotter"
titles_save = ["EI","ln(SSE)","Var","StDev","Best_Error","z","CDF","PDF","ei_term_1","ei_term_2"] 
# titles_save = ["EI"] 
##Choose theta_points to grab data from

ei_properties = pd.DataFrame(columns= ["BO_iter"] + [param_names_list[i] for i in range(len(param_names_list))] + 
                             [titles_save[i] for i in range(len(titles_save))] )

sep_fact_list = np.linspace(1.0,1.0,10)
row_num = 0
#Loop over number of EI components
for Bo_iter in range(tot_iter):   
    #Find theta_mesh associated with EI components
    path_mesh =  path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, 
                            obj, mesh_combo, Bo_iter, "theta_mesh", run, tot_iter, 
                            tot_runs, DateTime, sep_fact = sep_fact, 
                            is_figure = False, csv_end = "/" + "theta_mesh", 
                            normalize = normalize) + ".npy"
    theta_mesh = np.load(path_mesh)  
    theta_mesh_cols = theta_mesh.T.reshape(theta_mesh.shape[1]**2, theta_mesh.shape[0])
    theta_interest = np.array([[0,-2.0], [0,2.0], [0,0], [1,-1.0]])
    theta_interest = theta_mesh_cols
    num_interest_theta = theta_interest.shape[0]
    for m in range(num_interest_theta):
        #Find index of theta closest to the specified array
        distances = np.linalg.norm(theta_mesh_cols - theta_interest[m], axis=1)
        # Find the index of the closest point
        closest_index = np.argmin(distances)
        #Save pertinent values to an array
        theta_mesh_col = theta_mesh_cols[closest_index]
        row_num += 1
        for j in range(len(titles_save)):   
            save_array = titles_save[j]
            
            #Find value of EI component
            path =  path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, 
                                    obj, mesh_combo, Bo_iter, save_array, run, tot_iter, 
                                    tot_runs, DateTime, sep_fact = sep_fact, 
                                    is_figure = False, csv_end = "/" + save_array, 
                                    normalize = normalize) + ".npy"
            
            component = np.load(path)
            component = component.T.reshape(-1,1)
            for k in range(len(param_names_list)):  
                ei_properties.loc[row_num, param_names_list[k]] = theta_mesh_col[k]
                ei_properties.loc[row_num, "BO_iter"] = Bo_iter +1
                
                if len(component)-1 >= closest_index:
                    ei_properties.loc[row_num, save_array] = component[closest_index,0]
                else:
                    ei_properties.loc[row_num, save_array] = component[0,0]

In [72]:
print(ei_properties[0:12])

   BO_iter  \theta_1  \theta_2        EI   ln(SSE)       Var     StDev  \
1        1 -1.996318 -1.995411  0.008717  2.716194  0.554721  0.744796   
2        1 -1.996318 -1.785445  0.005521  2.849124  0.557536  0.746683   
3        1 -1.996318 -1.575479  0.004564  2.926844  0.573306   0.75717   
4        1 -1.996318 -1.365512  0.003779  2.981546  0.575624  0.758699   
5        1 -1.996318 -1.155546  0.002402  3.048942  0.547148  0.739694   
6        1 -1.996318  -0.94558  0.000923  3.156403  0.488109  0.698648   
7        1 -1.996318 -0.735614  0.000156  3.318091  0.409782  0.640142   
8        1 -1.996318 -0.525647  0.000006  3.537273  0.320505  0.566132   
9        1 -1.996318 -0.315681       0.0  3.810791  0.224765  0.474094   
10       1 -1.996318 -0.105715       0.0  4.128346  0.136433  0.369368   
11       1 -1.996318  0.104252       0.0  4.465583  0.067951  0.260674   
12       1 -1.996318  0.314218       0.0  4.779525   0.01802  0.134238   

   Best_Error          z       CDF   

In [73]:
CSV_name = "ei_properties/" + Method + "/lenscl_" + str(set_lengthscale) + "/all_theta_mesh"
# save_csv(ei_properties, CSV_name, ext='csv', verbose=False)

'ei_properties/1B/lenscl_1/all_theta_mesh.csv'

### Define  Median Objective function value over all separation factors, runs, and iters

In [ ]:
#WORKS

csv_end = ".csv" #For obj
fxn = "plot_obj"

median_obj =[]
sep_fact_list = np.linspace(0.1,1,10)
for k in range(len(sep_fact_list)):  
    path = "../" + path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
          title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                 sep_fact = sep_fact_list[k], is_figure = is_figure, csv_end = ".csv")
    # print(path)
    obj_array = csv_to_array(path)
    all_obj_list = []
    for i in range(tot_runs): 
        #Plot data
        obj_df_run = pd.DataFrame(data = obj_array[i])
        obj_df_i = obj_df_run.loc[(abs(obj_df_run) > 1e-6).any(axis=1),0]
        obj_no_zero_arr = list(np.array(obj_df_i).flatten())
        all_obj_list += obj_no_zero_arr
    median_obj.append(np.median(all_obj_list))

median_obj = np.array(median_obj)
median_obj

## Define the median number of iterations, the best sse for each sf, and the run and iter associated with it

In [ ]:
#WORKS

sep_fact_list = np.linspace(0.1,1,10)
med_iter_list = []
sse_best_list = []
iter_best_num_list = []
iter_median_num_list = []
run_best_num_list = []
median_obj_min =[]
for k in range(len(sep_fact_list)):  
    csv_end = ".csv" #For obj
    fxn = "plot_obj"

    path = "../"+ path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
              title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                     sep_fact = sep_fact_list[k], is_figure = is_figure, csv_end = ".csv")
    # print(path)
    obj_abs_min = csv_to_array(path)

    iter_max = []
    run_best = []
    iter_best = []
    sse_best = []
    for i in range(tot_runs):
        obj_min_df_run = pd.DataFrame(data = obj_abs_min[i])
        obj_min_df_i = obj_min_df_run.loc[(abs(obj_min_df_run) > 1e-6).any(axis=1),0]
#         print(obj_min_df_i)
        obj_min_no_zero_arr = list(np.array(obj_min_df_i).flatten())
        median_obj_min.append( np.min(obj_min_no_zero_arr) )
#         print(np.min(obj_min_no_zero_arr))
        sse_best.append(np.amin(obj_min_df_i))
        iter_max.append(len(obj_min_df_i))
        min_iter = np.where(np.amin(obj_min_df_i) == obj_min_df_i)
        #Not sure how to fix but sparse grid thetas are weird because the first theta is 0 for some reason for runs 5,6,7
#         if sparse_grid == True:
#             if i ==4 or i==5 or i == 6:
#                 iter_best.append(min_iter[0][0]+1)
#             else:
        iter_best.append(min_iter[0][0])
        run_best.append(i)
#     print(iter_max)    
    iter_median_num_list.append(np.median(iter_max))
    iter_best_act = np.array(iter_best) + 1
    run_best_act = np.array(run_best) + 1 
    sse_best_num = np.amin(sse_best)
    sse_best_list.append(sse_best_num)
    iter_best_ind = sse_best.index(sse_best_num)
    run_best_list = run_best_act[iter_best_ind]
    iter_best_num = iter_best_act[iter_best_ind]
    median_iters = np.median(iter_best_act)
    med_iter_list.append(median_iters)
    iter_best_num_list.append(iter_best_num)
    run_best_num_list.append(run_best_list)
# print(len(median_obj_min))
# print(median_obj_min)
median_obj_min_num = np.median(np.array(median_obj_min))
    
print(med_iter_list)
print(sse_best_list)
print(iter_best_num_list)
print(iter_median_num_list)
print(run_best_num_list)
print(np.exp(median_obj_min_num))
    

## Find the theta values associated with the lowest SSE value for each SF

In [ ]:
sep_fact_list = np.linspace(0.1,1,10)
theta_Opt = np.zeros((len(sep_fact_list),d))

for k in range(len(sep_fact_list)):
    Theta_array = []
    fxn = "plot_Theta"
    for i in range(len(param_dict.keys())):
        csv_end = "/"+param_dict[i]+ ".csv" #For Thetas #Need to ensure this saves correctly

        path_i = "../" + path_name(emulator, ep, sparse_grid, fxn, set_lengthscale, t, obj, mesh_combo = None, bo_iter= None, 
                  title_save = None, run = None, tot_iter=tot_iter, tot_runs=tot_runs, DateTime = DateTime, 
                         sep_fact = sep_fact_list[k], is_figure = is_figure, csv_end = csv_end)
        path_i = path_i.replace("Param_Conv/\\theta_"+str(i+1), "Theta_Conv/Theta_Conv_"+str(i+1))
#         print(path_i)
        Theta_array.append( csv_to_array(path_i) )

    Theta_array = np.array(Theta_array)
    #Not sure how to fix but sparse grid thetas are weird because of mismatched 0 columns/rows for k>5
    Theat_Opt = Theta_array[:, iter_best_num_list[k]-1,  run_best_num_list[k]-1]
    theta_Opt[k] = Theat_Opt
print(theta_Opt)

## Create Pandas DF

In [ ]:
#Create table arrays using pandas
df = pd.DataFrame(list(zip(theta_Opt[:,0], theta_Opt[:,1], sse_best_list, median_obj,iter_best_num_list,iter_median_num_list)),
               columns =['Theta 1', 'Theta 2', 'Best SSE', "Median SSE", "Best Eval #", "Median Eval #"])

In [ ]:
print(df)